In [7]:
from moviepy.editor import AudioFileClip, concatenate_audioclips
from scipy.io import wavfile
import numpy as np
import itertools
import os

print('OK !')

OK !


In [8]:
def improveloop(x0, a, b, sr=44100, w1=0.010, w2=0.100):
    """
    Input:  (a, b) is a loop
    Output: (a, B) is a better loop 
            distance (the less the distance the better the loop)
    This function moves the loop's endpoint b to B (up to 100 ms further) such that (a, B) is a "better" loop, i.e. sum((x0[a:a+10ms] - x0[B:B+10ms])^2) is minimal
    """

    W1 = int(w1*sr)
    W2 = int(w2*sr)
    x = x0[a:a+W1]
    y = x0[b:b+W2]
    delta = np.sum(x**2) - 2*np.correlate(y, x) + np.correlate(y**2, np.ones_like(x))
    K = np.argmin(delta)
    B = K + b
    distance = delta[K]

    return a, B, distance

def seamless_loop(wave_file, shortest_th = 0):
    sr, x = wavfile.read(wave_file)
    x0 = x if x.ndim == 1 else x[:, 0]
    x0 = np.asarray(x0, dtype=np.float32)
    
    error = False
    times = 0
    while not error:
        times+=1
        try:
            A = [int((2 + 0.25 * k) * sr) for k in range(times)]
            dist = np.inf
            for a, b in itertools.product(A, A):  # cartesian product: pairs (a, b) of points on the grid
                if b - a < 1 * sr:
                    continue
                a, B, d = improveloop(x0, a, b, sr=sr)
                if d < dist:
                    aa = a
                    BB = B
                    dist = d
        except:
            error = True

    valid_times = times-1
    d_dict = {}
    d_list = []

    A = [int((2 + 0.25 * k) * sr) for k in range(valid_times)]
    dist = np.inf
    for a, b in itertools.product(A, A):  # cartesian product: pairs (a, b) of points on the grid
        if b - a < 1 * sr:
            continue
        a, B, d = improveloop(x0, a, b, sr=sr)
#         print( 'Loop (%.3f, %.3f) improved to (%.3fs, %.3fs), distance: %i' % 
#             (a * 1.0 / sr, b * 1.0 / sr, a * 1.0 / sr, B * 1.0 / sr, d))
        d_list.append(d)
        d_dict[d] = (a * 1.0 / sr,B * 1.0 / sr)

        if d < dist:
            aa = a
            BB = B
            dist = d
    d_list.sort()
    return d_dict[d_list[shortest_th]]
print('OK !')

OK !


In [9]:
def wave_extraction(start_sec, end_sec, export_file, seconds, volume_up = 1):
    audio_file = "data/Neorhythm - User Guide - How to use Test Tube.webm"
    audioclip = AudioFileClip(audio_file)

    enhance_mental_capacity = audioclip.subclip(start_sec, end_sec)
    # enhance_mental_capacity.preview()
    enhance_mental_capacity.write_audiofile("data/temp.wav")
    loop_interval = seamless_loop("data/temp.wav")
    os.remove("data/temp.wav")

    seemless_enhance_mental_capacity = enhance_mental_capacity.subclip(loop_interval[0], loop_interval[1])
#     seemless_enhance_mental_capacity.preview()
    loop_no = int((seconds / seemless_enhance_mental_capacity.duration) + 1)
    loop_clip = [seemless_enhance_mental_capacity for each in range(loop_no)]
    full_clip = concatenate_audioclips(loop_clip).volumex(volume_up)

    full_clip.write_audiofile(export_file)
    audioclip.close()
    print('Export Completed !')
print('OK !')

OK !


In [10]:
#enhance_mental_capacity
%time wave_extraction(56.0, 62.5,"data/30min_enhance_mental_capacity.mp3", 1800)

MoviePy - Writing audio in data/temp.wav
MoviePy - Done.


chunk:   0%|          | 7/39722 [00:00<09:56, 66.61it/s, now=None]

MoviePy - Writing audio in data/30min_enhance_mental_capacity.mp3


MoviePy - Done.
Export Completed !
CPU times: user 10min 59s, sys: 22 s, total: 11min 21s
Wall time: 12min 22s


In [11]:
#energy_and_vitality
%time wave_extraction(66.4, 71.8,"data/30min_energy_and_vitality.mp3", 1800)

MoviePy - Writing audio in data/temp.wav


MoviePy - Done.


chunk:   0%|          | 10/39712 [00:00<07:15, 91.14it/s, now=None]

MoviePy - Writing audio in data/30min_energy_and_vitality.mp3


MoviePy - Done.
Export Completed !
CPU times: user 9min 49s, sys: 25.4 s, total: 10min 15s
Wall time: 12min 44s


In [12]:
#deep_relaxation
%time wave_extraction(76.5, 82.5,"data/30min_deep_relaxation.mp3", 1800)

MoviePy - Writing audio in data/temp.wav


MoviePy - Done.


chunk:   0%|          | 8/39703 [00:00<08:16, 79.95it/s, now=None]

MoviePy - Writing audio in data/30min_deep_relaxation.mp3


MoviePy - Done.
Export Completed !
CPU times: user 10min 9s, sys: 20.7 s, total: 10min 29s
Wall time: 11min 57s


In [ ]:
#improved_sleep
%time wave_extraction(87, 92,"data/20min_improved_sleep.mp3", 1200)

In [ ]:
#pain_control
%time wave_extraction(97.5, 100.7,"data/40min_pain_control.mp3", 2400)

In [13]:
#meditation_theta
%time wave_extraction(106, 111,"data/30min_meditation_theta.mp3", 1800)

chunk:   0%|          | 0/111 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in data/temp.wav


MoviePy - Done.


chunk:   0%|          | 0/39712 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in data/30min_meditation_theta.mp3


MoviePy - Done.
Export Completed !
CPU times: user 11min 59s, sys: 20.7 s, total: 12min 20s
Wall time: 12min 13s


In [14]:
#meditation_calming_synchronization
%time wave_extraction(117, 121,"data/30min_meditation_calming_synchronization.mp3", 1800)

MoviePy - Writing audio in data/temp.wav
MoviePy - Done.


chunk:   0%|          | 0/39691 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in data/30min_meditation_calming_synchronization.mp3


MoviePy - Done.
Export Completed !
CPU times: user 12min 8s, sys: 21.4 s, total: 12min 29s
Wall time: 12min 31s


In [ ]:
#emptyclip
emptyclip = AudioFileClip('data/5-minutes-of-silence.mp3')
emptyclip.write_audiofile('data/5min_empty.mp3')
emptyclip.close()

In [2]:
improved_sleep_20min = AudioFileClip('data/20min_improved_sleep.mp3')
emptyclip_5min = AudioFileClip('data/5min_empty.mp3')

In [3]:
flag_8hr = False
loop_clip = [improved_sleep_20min]
while not flag_8hr:
    if concatenate_audioclips(loop_clip).duration < 28800:
        loop_clip = loop_clip + [emptyclip_5min] + [improved_sleep_20min]
    else:
        flag_8hr = True


In [4]:
full_clip = concatenate_audioclips(loop_clip).subclip(0, 28800)
full_clip.duration

28800

In [5]:
%time full_clip.write_audiofile('data/8hr_improved_sleep.mp3')
emptyclip_5min.close()
improved_sleep_20min.close()

chunk:   0%|          | 42/635041 [00:00<26:06, 405.37it/s, now=None]

MoviePy - Writing audio in data/8hr_improved_sleep.mp3


MoviePy - Done.
CPU times: user 11min 15s, sys: 34.9 s, total: 11min 50s
Wall time: 23min 6s
